# Exercise 3.z Trajectories

This is a short exercise showing how you can create lines colored according to a 3rd value.

It is similar to an exercise prepared by Nicolas Piaget in the c2sm python workshop.

It uses data calculated with the `lagranto` tool. `lagranto` is part of the Lagrangian Analysis Tool, [Sprenger and Wernli, 2015](https://www.geosci-model-dev.net/8/2569/2015/gmd-8-2569-2015.html).

In [ ]:
import cartopy.crs as ccrs
import cartopy.util as cutil
import cartopy.feature as cfeature

import matplotlib.pyplot as plt
import numpy as np
import scipy

import seaborn as sns
import xarray as xr

%matplotlib inline

In [ ]:
import utils

## Load data

Code from the Traj tool in the [Atmospheric Dynamics Group at ETH Zurich](http://www.iac.ethz.ch/groups/wernli).

In [ ]:
fN = './../data/lsl_20070119_12_ana_48'

def read_trajectories(fN):

    open_file = open(fN, 'r')
    file_lines = open_file.readlines()
    nvariables = file_lines[2].strip().split()
    head = file_lines[0].split()

    nhead=5

    dtypes = ['f8' for var in nvariables]
    array = scipy.genfromtxt(fN,
                                 dtype=dtypes,
                                 names=nvariables,
                                 skip_header=nhead,
                                 missing_values=-999.99)

    timestep = float(array[1][0]) - float(array[0][0])
    period = float(array[-1][0]) - float(array[0][0])
    ntime = int(1 + scipy.around(period / timestep))
    ntra = int(array.size / ntime)

    # reshape traj file
    return scipy.reshape(array, (ntra, ntime))
    
    
traj = read_trajectories(fN)

### Exercise

 * plot the latitude and longitude of the trajectories
 
> each row in `traj` corresponds to one trajectory

In [ ]:
lon = traj['lon']
lat = traj['lat']

# code here



### Solution

In [ ]:
f = plt.figure()
ax = plt.axes(projection=ccrs.PlateCarree())

ax.coastlines()

lon = traj['lon']
lat = traj['lat']

n_tra = len(lon)
for i in range(len(lon)):
    ax.plot(lon[i], lat[i], color='0.5')


You can also directly plot all the lines:

In [ ]:
f = plt.figure()
ax = plt.axes(projection=ccrs.PlateCarree())

ax.coastlines()

lon = traj['lon']
lat = traj['lat']

ax.plot(lon.T, lat.T, color='0.5');

## colorline

There is no direct way, to color lines in pyplot. Therefore we start an internet search and find the following [notebook](http://nbviewer.jupyter.org/github/dpsanders/matplotlib-examples/blob/master/colorline.ipynb).

So we copy the code from there

In [ ]:
from matplotlib.collections import LineCollection
from matplotlib.colors import ListedColormap, BoundaryNorm


# Data manipulation:
def make_segments(x, y):
    '''
    Create list of line segments from x and y coordinates, in the correct format for LineCollection:
    an array of the form   numlines x (points per line) x 2 (x and y) array
    '''

    points = np.array([x, y]).T.reshape(-1, 1, 2)
    segments = np.concatenate([points[:-1], points[1:]], axis=1)
    
    return segments


# Interface to LineCollection:

def colorline(x, y, z=None, cmap=plt.get_cmap('copper'), norm=plt.Normalize(0.0, 1.0), linewidth=3, alpha=1.0):
    '''
    Plot a colored line with coordinates x and y
    Optionally specify colors in the array z
    Optionally specify a colormap, a norm function and a line width
    '''
    
    # Default colors equally spaced on [0,1]:
    if z is None:
        z = np.linspace(0.0, 1.0, len(x))
           
    # Special case if a single number:
    if not hasattr(z, "__iter__"):  # to check for numerical input -- this is a hack
        z = np.array([z])
        
    z = np.asarray(z)
    
    segments = make_segments(x, y)
    lc = LineCollection(segments, array=z, cmap=cmap, norm=norm, linewidth=linewidth, alpha=alpha)
    
    ax = plt.gca()
    ax.add_collection(lc)
    
    return lc

### Exercise

 * try to get colorline to work
 * you will need to use set_extent
 * add a colorbar
 * bonus: use `make_axes_locatable` to ensure the colorbar has the right height

In [ ]:
from mpl_toolkits.axes_grid1 import make_axes_locatable

print(lon.min(), lon.max())
print(lat.min(), lat.max())

In [ ]:
# code here


### Solution

In [ ]:
f = plt.figure()
ax = plt.axes(projection=ccrs.PlateCarree())

ax.coastlines()

lc = colorline(lon.T, lat.T)

plt.draw()

ax.set_extent([-90, 0, 20, 80], ccrs.PlateCarree())

# create axes that has the right size
divider = make_axes_locatable(ax)
cbax = divider.append_axes('right', size="6.5%", pad=0.1, axes_class=plt.Axes)

# create colorbar in this axes
cbar = plt.colorbar(lc, cax=cbax, orientation='vertical', extend='both')

### Exercise

 * color the lines according to the potential temperature (`TH = traj['TH']`)
 * what do you have to choose for `norm=plt.Normalize(min, max)`?
 * use the Reds colormap
 
Note
> you will have to loop through lat, lon and TH 
 

In [ ]:
f = plt.figure()
ax = plt.axes(projection=ccrs.PlateCarree())

ax.coastlines()

TH = traj['TH']

n_tra = len(lon)

# adjust colorline // add loop
lc = colorline(lon.T, lat.T)

ax.set_extent([-90, 0, 20, 80], ccrs.PlateCarree())

# create axes that has the right size
divider = make_axes_locatable(ax)
cbax = divider.append_axes('right', size="6.5%", pad=0.1, axes_class=plt.Axes)

# create colorbar in this axes
cbar = plt.colorbar(lc, cax=cbax, orientation='vertical', extend='both')

### Solution

Le't also add ticks and ticklabels. As this is a PlateCarree map, we can use the ordinary mechanism.

In [ ]:
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter

In [ ]:
f = plt.figure()
ax = plt.axes(projection=ccrs.PlateCarree())

ax.coastlines()

TH = traj['TH']

n_tra = len(lon)

norm = plt.Normalize(TH.min(), TH.max())
cmap = plt.get_cmap('Reds')

for i in range(n_tra):
    lc = colorline(lon[i], lat[i], TH[i], norm=norm , linewidth=1, cmap=cmap)


ax.set_extent([-90, 0, 20, 80], ccrs.PlateCarree())

# create axes that has the right size
divider = make_axes_locatable(ax)
cbax = divider.append_axes('right', size="4%", pad=0.1, axes_class=plt.Axes)

# create colorbar in this axes
cbar = plt.colorbar(lc, cax=cbax, orientation='vertical', extend='both')



# set ticks
tick_lon = np.arange(-90, 1, 15)
tick_lat = np.arange(20, 81, 10)

# set the ticks
ax.set_xticks(tick_lon, crs=ccrs.PlateCarree());
ax.set_yticks(tick_lat, crs=ccrs.PlateCarree());

# format the ticks as e.g 60°W
ax.xaxis.set_major_formatter(LongitudeFormatter())
ax.yaxis.set_major_formatter(LatitudeFormatter())